<a href="https://colab.research.google.com/github/niikun/DLentry/blob/main/lanchain_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [3]:
import os
from google.colab import userdata

In [4]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install langchain

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [12]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)
cot_template= """以下の質問に回答してください。
質問:{question}
ステップバイステップで考えてください。
"""

cot_prompt = PromptTemplate(template=cot_template,
                            input_variables=["question"],
                            )

In [13]:
from langchain import LLMChain

In [15]:
cot_chain = LLMChain(llm=chat,prompt=cot_prompt)
cot_chain.run(question="私は10個のリンゴを買いました。隣人に2つ、友達に2個渡しました。それから5つリンゴを買って、1つ食べました。残りは何個ですか？")

'1. 最初に10個のリンゴを買いました。\n2. 隣人に2つ渡しましたので、残りは10 - 2 = 8個です。\n3. 友達に2個渡しましたので、残りは8 - 2 = 6個です。\n4. さらに5つリンゴを買いましたので、残りは6 + 5 = 11個です。\n5. 1つリンゴを食べましたので、残りは11 - 1 = 10個です。\n\nしたがって、残りのリンゴは10個です。'

In [17]:
summarize_template = """以下の文章を結論だけ、一言で要約してください.
{input}
"""

sum_prompt=PromptTemplate(template=summarize_template,input_variables=["input"])

sumchain=LLMChain(llm=chat,prompt=sum_prompt)

In [18]:
sumchain.run(input="1. 最初に10個のリンゴを買いました。\n2. 隣人に2つ渡しましたので、残りは10 - 2 = 8個です。\n3. 友達に2個渡しましたので、残りは8 - 2 = 6個です。\n4. さらに5つリンゴを買いましたので、残りは6 + 5 = 11個です。\n5. 1つリンゴを食べましたので、残りは11 - 1 = 10個です。\n\nしたがって、残りのリンゴは10個です")

'残りのリンゴは10個。'

In [21]:
from langchain.chains import SimpleSequentialChain
cot_sum_chain = SimpleSequentialChain(chains=[cot_chain,sumchain])
result=cot_sum_chain("私は10個のリンゴを買いました。隣人に2つ、友達に2個渡しました。それから5つリンゴを買って、1つ食べました。残りは何個ですか？")
print(result["output"])

最終的に残るリンゴは10個です。


In [7]:
# class Recipe(BaseModel):
#   ingredients:list[str]=Field(description="ingredients of the dish")
#   steps:list[str]=Field(description="steps to make the dish")

# output_parser = PydanticOutputParser(pydantic_object=Recipe)
# # format_instructions=output_parser.get_format_instructions()

# template = """あなたは{genre}料理のプロです。レシピを考えてください。
# {format_instructions}
# 料理名:{dish}
# """

# prompt = PromptTemplate(
#     template=template,
#     input_variables=["dish","genre"],
#     partial_variables={"format_instructions":output_parser.get_format_instructions()})

# chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

In [ ]:
# from langchain import LLMChain
# chain = LLMChain(prompt=prompt,llm=chat,output_parser=output_parser)

# recipe = chain.run(dish="カレー",genre="和食")
# print(type(recipe))
# print(recipe)